# A Simple Growth Model

For a more thorough discussion of this model, please refer to the [QuantEcon lecture](http://quant-econ.net/jl/optgrowth.html). We choose to present this model because it is one that most people have seen which will allow us to focus on the programming material rather than the economics (though there are interesting economics in here as well).

In [1]:
using Interpolations
using Optim


## The Model

There is a single agent who values consumption according to:

$$\sum_{t=0}^{\infty} \beta^t u(c_t)$$

The agent produces the consumption good using period $t$ capital goods, $k_t$. His production follows $y_t = f(k_t)$. For the sake of simplicity, assume 100 percent depreciation so we have $k_{t+1} = y_t - c_t$. The corresponding Bellman equation is

$$V(k_t) = \max_{c_t} u(c_t) + \beta V(f(k_t) - c_t)$$

It turns out that if

$$f(k_t) = k_t^\alpha$$

and

$$u(c_t) = \log(c_t)$$

then we can write the value function in closed form by

$$V(k_t) = c_1 + c_2 \log (k_t)$$

with

$$c_1 = \frac{\log(1 - \alpha \beta)}{1 - \beta} + \frac{\log(\alpha \beta) \alpha \beta}{(1-\alpha \beta) (1 - \beta)}$$

and

$$c_2 = \frac{\alpha}{1 - \alpha \beta}$$

In [2]:
type GrowthModel
    # Parameters
    α::Float64
    β::Float64

    # Grid on capital
    kgrid::LinSpace
end

function GrowthModel(α=0.65, β=0.95, kmin=1e-2, kmax=2.0, nk=150)
    kgrid = linspace(kmin, kmax, nk)
    return GrowthModel(α, β, kgrid)
end

GrowthModel

In [3]:
function closed_form_value_function(gm::GrowthModel, k::Float64)

    # Pull out parameters
    α, β = gm.α, gm.β

    # Create coefficients
    c1_1 = log(1 - α*β) / (1-β)
    c1_2 = log(α*β)*α*β / ((1-α*β) * (1-β))
    c1 = c1_1 + c1_2
    c2 = α / (1 - α*β)

    return c1 + c2*log(k)
end

closed_form_value_function (generic function with 1 method)

In [4]:
gm = GrowthModel()
closed_form_value_function(gm, 1.0)

-34.78560754549537

In [5]:
function solve(gm::GrowthModel)
    # Pull out parameters
    α, β, kgrid = gm.α, gm.β, gm.kgrid

    # Allocate space for value function
    V = zeros(length(kgrid))
    V_old = copy(V)

    # Iterate until convergence
    tol, dist, it, maxiter = 1e-6, 10.0, 0, 500
    while (tol < dist) && (it < maxiter)
        # Create an interpolant over value function
        VI = scale(interpolate(V, BSpline(Linear()), OnGrid()), kgrid)
        # Iterate over all states
        for (ik, k) in enumerate(kgrid)
            # Production
            y = k^α

            # Objective function to minimize
            obj(c) = -(log(c) + β*VI[y - c])
            res = optimize(obj, 1e-8, y-1e-8, Brent())
            cstar = Optim.minimizer(res)

            # Update value
            V[ik] = log(cstar) + β*VI[y - cstar]
        end

        # Check distance update iteration
        it += 1
        dist = maxabs(V - V_old)
        copy!(V_old, V)
        mod(it, 50) == 0 ? println(it, "\t", dist) : nothing
    end

    return V
end

solve (generic function with 1 method)

In [6]:
V_vfi = solve(gm)

50	0.15568823362229267
100	0.011979427352237337
150	0.0009217567936019577
200	7.092460660373945e-5
250	5.457322501456474e-6


150-element Array{Float64,1}:
 -42.6597
 -41.1821
 -40.411 
 -39.8787
 -39.4759
 -39.1512
 -38.8787
 -38.6439
 -38.4378
 -38.2538
 -38.0874
 -37.9357
 -37.7965
   ⋮     
 -33.7391
 -33.7269
 -33.7149
 -33.7028
 -33.6909
 -33.6791
 -33.6673
 -33.6556
 -33.644 
 -33.6324
 -33.6211
 -33.6097

In [7]:
V_cf = [closed_form_value_function(gm, k) for k in gm.kgrid]

150-element Array{Float64,1}:
 -42.6114
 -41.1699
 -40.4014
 -39.8741
 -39.4723
 -39.1475
 -38.875 
 -38.6402
 -38.4339
 -38.25  
 -38.0841
 -37.9329
 -37.7941
   ⋮     
 -33.7374
 -33.7252
 -33.713 
 -33.701 
 -33.6891
 -33.6772
 -33.6654
 -33.6537
 -33.6421
 -33.6306
 -33.6191
 -33.6077

In [8]:
maxabs(V_vfi - V_cf)

0.04826642703308437